In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

# Arguments

In [2]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [3]:
pretrained = './log/style1/checkpoint.pth.tar'
resume = ''

In [4]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 400
start_epoch = 0
train_batch = 512
test_batch = 300
lr = 0.04
schedule = [150, 225]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

#
size = (128, 128)

best_acc = 0

In [5]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [6]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Dataset

In [7]:
test_dir = os.path.join(data_dir, 'test')

test_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

test_loader = DataLoader(datasets.ImageFolder(test_dir, test_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/checkpoint.pth.tar'


In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# # Resume
# if resume:
#     print('==> Resuming from checkpoint..')
#     checkpoint = os.path.dirname(resume)
# #     checkpoint = torch.load(resume)
#     resume = torch.load(resume)
#     best_acc = resume['best_acc']
#     start_epoch = resume['epoch']
#     model.load_state_dict(resume['state_dict'])
#     optimizer.load_state_dict(resume['optimizer'])
#     logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
# else:
#     logger = Logger(os.path.join(checkpoint, 'log.txt'))
#     logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

In [16]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in tqdm(enumerate(val_loader)):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    if epoch in schedule:
        state['lr'] *= gamma
        for param_group in optimizer.param_groups:
            param_group['lr'] = state['lr']

In [18]:
test_loss, test_acc = test(test_loader, model, criterion, 1, use_cuda)

200it [03:36,  1.46s/it]

200/200 Data:3.443 | Batch:3.575 | Total:0:03:36 | ETA:0:00:00 | Loss:0.005895469192910241 | top1:99.80500030517578


In [19]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
data_dir = '/media/data2/dataset/GAN_ImageData/PGGAN/128x128'

test_dir = os.path.join(data_dir, 'test_50')

test_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

test_loader = DataLoader(datasets.ImageFolder(test_dir, test_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

In [20]:
test_loss, test_acc = test(test_loader, model, criterion, 1, use_cuda)

126it [00:27,  2.79it/s]

126/126 Data:0.002 | Batch:0.425 | Total:0:00:28 | ETA:0:00:00 | Loss:4.261480209336573 | top1:49.99201202392578
